In [1]:

import pandas as pd
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
from nba_api.stats.static import teams
from nba_api.stats.endpoints import playbyplay
import numpy as np
import time
from statistics import multimode
import os

In [33]:
test_list = []
season_list = ['2000-01', '2001-02', '2002-03', '2003-04', '2004-05', '2005-06', '2006-07', '2007-08', '2008-09', \
              '2009-10', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', \
              '2018-19', '2019-20', '2020-21', '2021-22']


In [34]:
get_team_and_year(nba_teams, season_list, test_list)

(['2000-01',
  '2001-02',
  '2002-03',
  '2003-04',
  '2004-05',
  '2005-06',
  '2006-07',
  '2007-08',
  '2008-09',
  '2009-10',
  '2010-11',
  '2011-12',
  '2012-13',
  '2013-14',
  '2014-15',
  '2015-16',
  '2016-17',
  '2017-18',
  '2018-19',
  '2019-20',
  '2020-21',
  '2021-22'],
 [{'id': 1610612737,
   'full_name': 'Atlanta Hawks',
   'abbreviation': 'ATL',
   'nickname': 'Hawks',
   'city': 'Atlanta',
   'state': 'Atlanta',
   'year_founded': 1949},
  {'id': 1610612738,
   'full_name': 'Boston Celtics',
   'abbreviation': 'BOS',
   'nickname': 'Celtics',
   'city': 'Boston',
   'state': 'Massachusetts',
   'year_founded': 1946},
  {'id': 1610612739,
   'full_name': 'Cleveland Cavaliers',
   'abbreviation': 'CLE',
   'nickname': 'Cavaliers',
   'city': 'Cleveland',
   'state': 'Ohio',
   'year_founded': 1970},
  {'id': 1610612740,
   'full_name': 'New Orleans Pelicans',
   'abbreviation': 'NOP',
   'nickname': 'Pelicans',
   'city': 'New Orleans',
   'state': 'Louisiana',
   'ye

In [2]:
def get_team_and_year(nba_teams, season_list, sixth_man_df_list):
    # IF the Df exists
    if len(sixth_man_df_list) == 1:
        df = sixth_man_df_list[0]
        last_team = df.loc[len(df) - 1, 'Team']
        last_season = df.loc[len(df) - 1, 'Season']
        # Get team_list
        for i in range(len(nba_teams)):
            if last_team == nba_teams[i]['full_name']:
                used_team_list = nba_teams[i:]
        # Get correct season_list and update team_list if need be
        if last_season == '2021-22':
            used_season_list = season_list
            used_team_list = used_team_list[1:]
        else: 
            correct_index = season_list.index(last_season)
            used_season_list = season_list[(correct_index+1):]
    # If df does not exist
    else: 
        used_season_list = season_list
        used_team_list = nba_teams

    return used_season_list, used_team_list
 



In [21]:
test_list = [['Maloney'], ['Henderson'], ['Bowdler'], ['Mottola'], ['Mottola'], ['Mottola'], ['Wright'], ['Wright', 'Mottola', 'Maloney'], ['Wright', 'Mottola'], ['Wright'], ['Maloney'], ['Wright'], ['Glover'], ['Mottola', 'Maloney'], ['Wright'], ['Mottola', 'Maloney'], ['Crawford', 'Maloney'], ['Henderson'], ['Glover', 'Henderson'], ['Sanchez'], ['Mohammed', 'Johnson'], ['Johnson'], ['Mohammed', 'Kukoc'], ['Sanchez'], ['Mohammed', 'Kukoc'], ['Johnson'], ['Mottola', 'Glover', 'Mohammed'], ['Bowdler', 'Mottola'], ['Robinson', 'Henderson'], ['Henderson'], ['Glover'], ['Crawford'], ['Crawford'], ['McLeod', 'Henderson'], ['Crawford'], ['Crawford'], ['Smith'], ['Knight'], ['Henderson'], ['McLeod'], ['McLeod'], ['Crawford', 'Smith', 'Henderson'], ['Crawford'], ['Glover'], ['Colson', 'Glover', 'Henderson'], ['Henderson'], ['Glover'], ['Glover', 'Maloney'], ['Henderson'], ['Knight', 'Glover', 'Henderson'], ['Knight', 'Mottola'], ['Johnson'], ['Jackson'], ['Jackson', 'A. Johnson'], ['Mottola', 'Jackson'], ['Robinson'], ['Mottola', 'A. Johnson'], ['D. Johnson'], ['Mottola', 'D. Johnson'], ['McLeod'], ['Robinson'], ['Wright'], ['Robinson'], ['Mottola', 'Wright'], ['Robinson', 'Wright'], ['Mottola', 'Wright'], ['Wright'], ['D. Johnson'], ['D. Johnson'], ['A. Johnson', 'Wright'], ['Wright'], ['Terry', 'Wright', 'Mottola'], ['Wright'], ['Robinson'], ['Henderson'], ['A. Johnson', 'D. Johnson'], ['Wright'], ['Maloney'], ['Wright'], ['D. Johnson', 'A. Johnson'], ['Crawford'], ['Crawford']]

In [1]:
from sixthman import sixth_man_main
for i in range(100):
  for attempt in range(10):
    try:
      sixth_man_main()
    except:
      continue
    else:
      break
  else:
    print('Failed too many Times')


Good Update
here
  PCTIMESTRING  PERIOD              DESCRIPTION
0         3:36       1  SUB: Kukoc FOR Mohammed
1         3:36       1   SUB: Smith FOR Johnson
  PCTIMESTRING  PERIOD             DESCRIPTION
0         8:29       1  SUB: Kukoc FOR Johnson
  PCTIMESTRING  PERIOD                   DESCRIPTION
0         3:02       1  SUB: Mottola FOR Abdur-Rahim
  PCTIMESTRING  PERIOD                DESCRIPTION
0         9:57       1  SUB: Bowdler FOR Mohammed
  PCTIMESTRING  PERIOD                DESCRIPTION
0         5:49       1  SUB: Mottola FOR Mohammed
1         5:49       1    SUB: Vaughn FOR Johnson
  PCTIMESTRING  PERIOD             DESCRIPTION
0         3:50       1  SUB: Kukoc FOR Johnson
  PCTIMESTRING  PERIOD            DESCRIPTION
0         6:13       1  SUB: Vaughn FOR Terry
  PCTIMESTRING  PERIOD                  DESCRIPTION
0         2:29       1  SUB: Henderson FOR Mohammed
  PCTIMESTRING  PERIOD              DESCRIPTION
0         5:33       1  SUB: Vaughn FOR Johnson
  P

In [5]:
sixth_man_main()

Good
here
  PCTIMESTRING  PERIOD              DESCRIPTION
0         5:56       1  SUB: Maloney FOR Knight
  PCTIMESTRING  PERIOD                  DESCRIPTION
0         7:42       1  SUB: Henderson FOR Mohammed
  PCTIMESTRING  PERIOD                DESCRIPTION
0         5:27       1  SUB: Bowdler FOR Mohammed
  PCTIMESTRING  PERIOD              DESCRIPTION
0         7:47       1  SUB: Johnson FOR Glover
  PCTIMESTRING  PERIOD              DESCRIPTION
0         7:06       1  SUB: Maloney FOR Knight
  PCTIMESTRING  PERIOD                DESCRIPTION
0         7:36       1  SUB: Henderson FOR Wright
  PCTIMESTRING  PERIOD               DESCRIPTION
0         7:16       1  SUB: Wright FOR Mohammed
  PCTIMESTRING  PERIOD                DESCRIPTION
0         6:00       1  SUB: Wright FOR Henderson
1         6:00       1  SUB: Mottola FOR Mohammed
2         6:00       1    SUB: Maloney FOR Knight
  PCTIMESTRING  PERIOD                 DESCRIPTION
0         3:15       1    SUB: Wright FOR Mohamme

In [54]:
def return_mode_player(sixth_man_list):
  total_players_list = []
  for value in sixth_man_list:
      for x in value: total_players_list.append(x)
  return multimode(total_players_list)

areturn_mode_player(test_list)

ImportError: cannot import name 'main_sixth_man' from 'sixthman' (c:\Users\kaste\dataManipulation\6thManProject\sixthman.py)

In [42]:

def add_to_list(sixth_men_mode, sixth_man_df_list, season, team):
    for player in sixth_men_mode:
        sixth_man_df_list.append(pd.DataFrame({'Player': [player], 'Season': [season], 'Team': [team]}))
    return sixth_man_df_list

sixth_man_df_list = add_to_list(a, sixth_man_df_list, '2000-01', 'Atlanta Hawks')
pd.concat(sixth_man_df_list)


,Player,Season,Team
0,Wright,2000-01,Atlanta Hawks
0,Wright,2000-01,Atlanta Hawks
0,Wright,2000-01,Atlanta Hawks
0,Wright,2000-01,Atlanta Hawks
0,Wright,2000-01,Atlanta Hawks
0,Wright,2000-01,Atlanta Hawks
0,Wright,2000-01,Atlanta Hawks
0,Wright,2000-01,Atlanta Hawks
0,Wright,2000-01,Atlanta Hawks
0,Wright,2000-01,Atlanta Hawks


In [49]:
type(pd.DataFrame({}))

pandas.core.frame.DataFrame

In [34]:
sixth_man_df_list

,Player,Season,Team
0,Wright,2000-01,Atlanta Hawks
0,Wright,2000-01,Atlanta Hawks
